## Extension of the 5% pool for all tracers in paper 1

In [1]:
import cmocean as cmo
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import numpy as np
import seaborn as sns
import canyon_tools.readout_tools as rout
import canyon_tools.savitzky_golay as sg

import matplotlib.gridspec as gspec

import os
import sys
import warnings
warnings.filterwarnings("ignore")

In [2]:
sns.set_context('notebook')
sns.set_style('white')

In [ ]:
# Grid, state and tracers datasets of base case
grid_file = '/data/kramosmu/results/TracerExperiments/CNTDIFF/run38/gridGlob.nc'

grid_fileNoC = '/data/kramosmu/results/TracerExperiments/CNTDIFF/run38/gridGlob.nc'

state_file = '/data/kramosmu/results/TracerExperiments/CNTDIFF/run38/stateGlob.nc' 


ptracers_file = '/data/kramosmu/results/TracerExperiments/BARKLEY/run01/ptracersGlob.nc'

ptracers_fileNoC = '/data/kramosmu/results/TracerExperiments/BARKLEY/run02/ptracersGlob.nc'

In [ ]:
with Dataset(grid_file, 'r') as nbl:
    Z = nbl.variables['RC'][:]
    X = nbl.variables['X'][:]
    Y = nbl.variables['Y'][:]
    hFacC = nbl.variables['HFacC'][:]
    rA = nbl.variables['rA'][:]
    Depth = nbl.variables['Depth'][:]

hfac = np.ma.masked_values(hFacC, 0)
maskC = np.ma.getmask(hfac)

with Dataset(grid_fileNoC, 'r') as nbl:
    hFacCNoC = nbl.variables['HFacC'][:]

hfac = np.ma.masked_values(hFacCNoC, 0)
maskCNoC = np.ma.getmask(hfac)
 

with Dataset(state_file, 'r') as nbl:
    iters = nbl.variables['iter'][:]
 

In [ ]:
lib_path = os.path.abspath('/ocean/kramosmu/OutputAnalysis/outputanalysisnotebooks/PythonScripts/Paper1Figures/') # Add absolute path to my python scripts
sys.path.append(lib_path)

import canyon_records 
import nocanyon_records

records = canyon_records.main()
recordsNoC = nocanyon_records.main()

for ii in range(len(records)):
    print(ii,records[ii].name)
select_rec = [0,2,13,15,5,9,18]
    
records2 = [records[0],
            records[13],
            records[15],
            records[5],
            records[9],
            records[18],
            ]
recordsNoC2 = [recordsNoC[0],
               recordsNoC[13],
               recordsNoC[15],
               recordsNoC[5],
               recordsNoC[9],
               recordsNoC[18],
               ]


In [ ]:
def mask2DCanyon(bathy, sbdepth=-152.5):
    '''Mask out the canyon from the shelf.
    bathy : depths 2D array from the grid file
    sbdepth: shelf depth, always negative float 
    Returns mask'''
    
    bathyMasked = np.ma.masked_less(-bathy, -152.5)
    return(bathyMasked.mask)

def ConcAreaPool(Tr, hfac, ra, bathy, sbdepth=-152.5):
    '''Tr: Tracer field (nt,nz,ny,nx)
       hfac: fraction of open cell at center (nz,ny,nx)
       ra: array of cell horizontal areas (ny,nx)
       bathy : depths 2D array from the grid file (ny,nx)
       sbdepth: shelf break depth (negative value)
       
       RETURNS:
       ConcArea = concentration at cell closest to bottom times its area (nt,ny,nx)
       Conc = cocnetration near bottom (nt,ny,nx)'''
    
    ConcArea = np.empty((360,616))
    Conc = np.empty((360,616))
    ConcFiltered = np.empty((360,616))
    Area = np.empty((360,616))
    BottomInd = np.argmax(hfac[::-1,:,:]>0.0,axis=0) # start looking for first no-land cell from the bottom up.
    BottomInd = np.ones(np.shape(BottomInd))*89 - BottomInd # Get index of unreversed z axis
    
    for j in range(616):
        for i in range(360):
            TrBottom = Tr[BottomInd[i,j],i,j]
            if TrBottom > 0.0:
                ConcArea[i,j] = TrBottom*ra[i,j]
                Conc[i,j] = TrBottom
                Area[i,j] = ra[i,j]
            else:
                ConcArea[i,j] = np.NaN
                Conc[i,j] = np.NaN
                Area[i,j] = np.NaN
                    
        # Filter step noise
        ConcFiltered[:,j] = sg.savitzky_golay(Conc[:,j], 7,3) 
                
    maskShelf = mask2DCanyon(bathy, sbdepth)
    maskShelf = np.expand_dims(maskShelf,0) # expand along time dimension
    maskShelf = maskShelf + np.zeros(Conc.shape)
    
    return (ConcArea,
            np.ma.masked_array(ConcFiltered, mask=maskShelf),
            Area,
           )

def BottomConc(Tr, hfac, ra, bathy, sbdepth=-152.5):
    '''Tr: Tracer field (nt,nz,ny,nx)
       hfac: fraction of open cell at center (nz,ny,nx)
       ra: array of cell horizontal areas (ny,nx)
       bathy : depths 2D array from the grid file (ny,nx)
       sbdepth: shelf break depth (negative value)
       
       RETURNS:
       ConcArea = concentration at cell closest to bottom times its area (nt,ny,nx)
       Conc = cocnetration near bottom (nt,ny,nx)'''
    
    ConcArea = np.empty((360,616))
    Conc = np.empty((360,616))
    ConcFiltered = np.empty((360,616))
    Area = np.empty((360,616))
    BottomInd = np.argmax(hfac[::-1,:,:]>0.0,axis=0) # start looking for first no-land cell from the bottom up.
    BottomInd = np.ones(np.shape(BottomInd))*89 - BottomInd # Get index of unreversed z axis
    
    for j in range(616):
        for i in range(360):
            TrBottom = Tr[BottomInd[i,j],i,j]
            ConcArea[i,j] = TrBottom*ra[i,j]
            Conc[i,j] = TrBottom
            Area[i,j] = ra[i,j]
                    
        # Filter step noise
        ConcFiltered[:,j] = sg.savitzky_golay(Conc[:,j], 7,3) 
                
    maskShelf = mask2DCanyon(bathy, sbdepth)
    maskShelf = np.expand_dims(maskShelf,0) # expand along time dimension
    maskShelf = maskShelf + np.zeros(Conc.shape)
    
    return (ConcArea,
            np.ma.masked_array(ConcFiltered, mask=maskShelf),
            Area,
           )

def PlotPool(ax,ConcFilt,xslice,yslice,col):
    
    ax.axvline(X[xind_CS]/1000, color = '0.5', linestyle = '-', linewidth = 1)
    ax.axhline(Y[yind_AS]/1000, color = '0.5', linestyle = '-', linewidth = 1)
    
    BAC = ConcFilt[yslice,xslice]
    
    mesh = ax.contour(X[xslice]/1000,Y[yslice]/1000,BAC,[0.05], 
                      colors=col, 
                      )
    
    
    mesh = ax.contour(X[xslice]/1000,Y[yslice]/1000,BAC,[-0.05], 
                      colors=col, 
                      )
    
    SB = ax.contour(X[xslice]/1000,Y[yslice]/1000,
                    Depth[yslice,xslice],
                    [150.0],
                    colors='0.1',linewidths=[0.75] )
    ax.tick_params(axis='x', pad=1)
    ax.tick_params(axis='y', pad=1)
    ax.set_aspect(1)
    
def PlotAS(ax,Conc,xsl,zsl,col):
    print(np.shape(X[xsl]/1000),np.shape(Z[zsl]), np.shape(Conc))
   
    mesh = ax.contour(X[xsl]/1000,Z[zsl],Conc,[0.05], 
                      colors=col, 
                      )
    mesh = ax.contour(X[xsl]/1000,Z[zsl],Conc,[-0.05], 
                      colors=col, 
                      )
    
    ax.plot(X[xsl]/1000,-Depth[yind_AS,xsl],
           '-',
           color='0.5',
           linewidth=2,
           )
    ax.tick_params(axis='x', pad=1)
    ax.tick_params(axis='y', pad=1)
    
def PlotCS(ax,Conc,yslice,zslice,col):
    
    mesh = ax.contour(Y[yslice]/1000,Z[zslice],Conc,[0.05], 
                      colors=col, 
                      )
    mesh = ax.contour(Y[yslice]/1000,Z[zslice],Conc,[-0.05], 
                      colors=col, 
                      )
    
    ax.plot(Y[yslice]/1000,-Depth[yslice_CS, xind_CS],
           '-',
           color='0.5',
           linewidth=2,
           )
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
        
    ax.tick_params(axis='x', pad=1)
    ax.tick_params(axis='y', pad=1)

In [ ]:
# General input
nx = 616
ny = 360
nz = 90
nt = 19 # t dimension size 
yind_AS = 275
xind_CS = 260

xslice_AS = slice(120,500)
yslice_CS = slice(227,360)
zslice_AS = slice(0,30)

yslice = slice(225,360)
xslice = slice(120,500)

In [ ]:
for rec in records2:
    
    ptracers_file = '/data/kramosmu/results/TracerExperiments/%s/%s/ptracersGlob.nc' %(rec.exp_code,rec.run_num)
    with Dataset(ptracers_file, 'r') as nbl:
        
        ConcArea, rec.concBOT, Area = BottomConc(np.mean(nbl.variables['Tr1'][8:18,:,:,:], axis=0),hFacC,rA,Depth)
        rec.concAS = np.ma.array(np.nanmean(nbl.variables['Tr1'][8:18,zslice_AS,yind_AS,xslice_AS],0),
                                  mask=maskCNoC[zslice_AS,yind_AS,xslice_AS])
        rec.concCS = np.ma.array(np.nanmean(nbl.variables['Tr1'][8:18,zslice_AS,yslice_CS,xind_CS],0),
                                  mask=maskCNoC[zslice_AS,yslice_CS,xind_CS])
       
    print('done with tracer %s %s' %(rec.exp_code, rec.run_num) )

In [ ]:
for rec in records2:
    
    ptracers_file = '/data/kramosmu/results/TracerExperiments/%s/%s/ptracersGlob.nc' %(rec.exp_code,rec.run_num)
    with Dataset(ptracers_file, 'r') as nbl:
        
        ConcArea, rec.concBOT_ini, Area = BottomConc(nbl.variables['Tr1'][0,:,:,:],hFacC,rA,Depth)
        rec.concAS_ini = np.ma.array(nbl.variables['Tr1'][0,zslice_AS,yind_AS,xslice_AS],
                                  mask=maskCNoC[zslice_AS,yind_AS,xslice_AS])
        rec.concCS_ini = np.ma.array(nbl.variables['Tr1'][0,zslice_AS,yslice_CS,xind_CS],
                                  mask=maskCNoC[zslice_AS,yslice_CS,xind_CS])
       
       
    print('done with tracer %s %s' %(rec.exp_code, rec.run_num) )


In [ ]:
for rec, recNoC in zip(records2, recordsNoC2):
    
    ptracers_file = '/data/kramosmu/results/TracerExperiments/%s/%s/ptracersGlob.nc' %(rec.exp_code,rec.run_num)
    ptracers_fileNoC = '/data/kramosmu/results/TracerExperiments/%s/%s/ptracersGlob.nc' %(recNoC.exp_code,recNoC.run_num)
    
    with Dataset(ptracers_file, 'r') as nbl:
        with Dataset(ptracers_fileNoC, 'r') as noc:
            ConcArea, rec.concBOT_anom, Area = BottomConc(np.nanmean(nbl.variables['Tr1'][8:18,:,:,:]-
                                                          noc.variables['Tr1'][8:18,:,:,:],axis=0),hFacC,rA,Depth)
            rec.concAS_anom = np.ma.array(np.nanmean(nbl.variables['Tr1'][8:18,zslice_AS,yind_AS,xslice_AS]-
                                                     noc.variables['Tr1'][8:18,zslice_AS,yind_AS,xslice_AS],0),
                                          mask=maskCNoC[zslice_AS,yind_AS,xslice_AS])
            rec.concCS_anom = np.ma.array(np.nanmean(nbl.variables['Tr1'][8:18,zslice_AS,yslice_CS,xind_CS]-
                                                 noc.variables['Tr1'][8:18,zslice_AS,yslice_CS,xind_CS],0),
                                          mask=maskCNoC[zslice_AS,yslice_CS,xind_CS])
       
       
    print('done with tracer %s %s' %(rec.exp_code, rec.run_num) )

In [ ]:
f = plt.figure(figsize = (10.5,5.0)) # 19.0cm = 7.4in, 115cm = 4.52in

gs = gspec.GridSpec(1, 2, width_ratios=[1,0.4], wspace=0.05, hspace=0.05 )

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

f1 = plt.figure(figsize = (10.5,5.0)) # 19.0cm = 7.4in, 115cm = 4.52in

gs2 = gspec.GridSpec(1, 1,  width_ratios=[1,0.2], wspace=0.05, hspace=0.05 )

ax2 = plt.subplot(gs2[0])

ii = 0    

for rec in records2:
    PlotPool(ax2,(rec.concBOT_anom)/rec.concBOT_ini,xslice,yslice,col=sns.xkcd_rgb[rec.color2])

    PlotAS(ax0,(rec.concAS_anom[:,:]/rec.concAS_ini),xslice_AS,zslice_AS,col=sns.xkcd_rgb[rec.color2])
 
    PlotCS(ax1,(rec.concCS_anom[:,:]/rec.concCS_ini[:,:]),yslice_CS,zslice_AS,col=sns.xkcd_rgb[rec.color2])


ax0.set_ylabel('Depth (m)')
ax2.set_xlabel('Alongshelf distance (km)')
ax2.set_ylabel('Cross-shelf distance (km)')
ax1.set_xlabel('Cross-shelf distance (km)')
ax1.set_ylabel('Depth (m)')

ax1.legend(ncol=4, bbox_to_anchor=(-0.5,1) )

f1.savefig('pool_vert_5percent_dyn.eps', format='eps',bbox_inches='tight')
f2.savefig('pool_5percent_dyn', format='eps',bbox_inches='tight')


In [ ]:
yslice2 = slice(225,320)
xslice2 = slice(120,500)
zslice_AS2 = slice(5,20)
zslice_CS2 = slice(5,30)

f1 = plt.figure(figsize = (11,4)) # 19.0cm = 7.4in, 115cm = 4.52in

gs = gspec.GridSpec(1, 2, width_ratios=[1, 0.8], wspace=0.0, hspace=0.05 )
gs0 = gspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs[1],width_ratios=[1,0.8], wspace=0.25)

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs0[0])

f2, ax2 = plt.subplots(1,1,figsize = (11,4.0)) # 19.0cm = 7.4in, 115cm = 4.52in

ii = 0    

for rec in records2:
    PlotPool(ax2,(rec.concBOT_anom)/rec.concBOT_ini,xslice2,yslice2,col=sns.xkcd_rgb[rec.color2])

    PlotAS(ax0,(rec.concAS_anom[zslice_AS2, :]/rec.concAS_ini[zslice_AS2, :]),xslice_AS,zslice_AS2,col=sns.xkcd_rgb[rec.color2])
 
    PlotCS(ax1,(rec.concCS_anom[zslice_CS2,:]/rec.concCS_ini[zslice_CS2,:]),yslice_CS,zslice_CS2,col=sns.xkcd_rgb[rec.color2])



ax0.set_ylabel('Depth (m)')
ax2.set_xlabel('Alongshelf distance (km)')
ax0.set_xlabel('Alongshelf distance (km)')

ax2.set_ylabel('Cross-shelf distance (km)')
ax1.set_xlabel('Cross-shelf distance (km)')
#ax1.set_ylabel('Depth (m)')


f1.savefig('pool_vert_5percent_dyn.eps', format='eps',bbox_inches='tight')
f2.savefig('pool_5percent_dyn.eps', format='eps',bbox_inches='tight')


## Bottom concentrations

In [ ]:
for rec, recNoC in zip(records2, recordsNoC2):
    ptracers_file = '/data/kramosmu/results/TracerExperiments/%s/%s/ptracersGlob.nc' %(rec.exp_code,rec.run_num)
    ptracers_fileNoC = '/data/kramosmu/results/TracerExperiments/%s/%s/ptracersGlob.nc' %(recNoC.exp_code,recNoC.run_num)
    rec.BCA = np.empty((nt,ny,nx))
    rec.BC = np.empty((nt,ny,nx))
    with Dataset(ptracers_file, 'r') as nbl:
        with Dataset(ptracers_fileNoC, 'r') as noc:
            for ii in range(nt):
                rec.BCA[ii,:,:], dummy, rec.AreaTot1 = BottomConc(noc.variables['Tr1'][ii,:,:,:], hFacCNoC,rA,Depth)
                rec.BC[ii,:,:], dummy, rec.AreaTot2 = BottomConc(nbl.variables['Tr1'][ii,:,:,:],hFacCNoC,rA,Depth)


In [ ]:
fig, (ax1,ax3) = plt.subplots(1,2,figsize=(10,4))

for rec in records2:
    ax1.plot(np.arange(19)/2, (((np.nansum(np.nansum(rec.BC[:,227:310,120:463],axis=1),axis=1)/
                                (np.nansum(rec.AreaTot2[227:310,120:463])))/
                               (np.nansum(rec.BC[0,227:310,120:463])/(np.nansum(rec.AreaTot2[227:310,120:463]))))-
                               ((np.nansum(np.nansum(rec.BCA[:,227:310,120:463],axis=1),axis=1)/
                                 (np.nansum(rec.AreaTot1[227:310,120:463])))/
                                (np.nansum(rec.BCA[0,227:310,120:463])/(np.nansum(rec.AreaTot1[227:310,120:463]))))),
             color=sns.xkcd_rgb[rec.color2], label=rec.label2)

    ax3.plot(np.arange(19)/2, ((np.nansum(np.nansum(rec.BC[:,227:310,120:463],axis=1),axis=1)/(np.nansum(rec.AreaTot2[227:310,120:463])))/
            (np.nansum(rec.BC[0,227:310,120:463])/(np.nansum(rec.AreaTot2[227:310,120:463])))),
             color=sns.xkcd_rgb[rec.color2])

ax1.set_xlim(0,7)
#ax2.set_xlim(0,7)
ax3.set_xlim(0,7)

ax1.set_ylim(-0.05, 0.33)
ax1.legend(bbox_to_anchor=(1.7, -0.1), ncol=4)
ax3.set_ylabel('Normalized bottom concentration')
ax1.set_ylabel('Bottom concentration anomaly')
#ax2.set_ylabel('Area ( $10^9$ m$^2$)')
fig.savefig('bottom_conc_Dyn.eps', format='eps',bbox_inches='tight')

